<a href="https://colab.research.google.com/github/tirthasheshpatel/semester6_practicals/blob/main/dl/Practical-6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("tensorflow").addHandler(logging.NullHandler(logging.ERROR))

In [3]:
data = "I am Tirth Patel.\nI am a blasphemous coder.\nI always upload my code on Google Drive.\nI don't use GitHub.\nI think coders who use GitHub are dumb.\nI am smart."
text = data.split('\n')

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$*&()*+,-/:;<=>?@[\]^_{}~')
tokenizer.fit_on_texts(text)

In [5]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

25

In [6]:
sequences = tokenizer.texts_to_sequences(text)
l = len(sequences)

In [7]:
max_len = max([len(seq) for seq in sequences])
max_len

8

In [8]:
X = []
y = sequences
for sq in sequences:
    X.append(sq[:-1])

In [9]:
max_len = max([len(seq) for seq in sequences])

In [10]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_len+1)
y = tf.keras.preprocessing.sequence.pad_sequences(y, maxlen=max_len+1)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [11]:
model = tf.keras.models.Sequential([
                                       tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=50),
                                       tf.keras.layers.SimpleRNN(50, return_sequences=True),
                                       tf.keras.layers.Dense(vocab_size, activation="softmax")
                                   ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1250      
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 50)          5050      
_________________________________________________________________
dense (Dense)                (None, None, 25)          1275      
Total params: 7,575
Trainable params: 7,575
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer="RMSprop", loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X, y, epochs=200)

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 3.2542 - accuracy: 0.0185
Epoch 2/200
1/1 [==============================] - 0s 11ms/step - loss: 3.1721 - accuracy: 0.1296
Epoch 3/200
1/1 [==============================] - 0s 14ms/step - loss: 3.1123 - accuracy: 0.3519
Epoch 4/200
1/1 [==============================] - 0s 14ms/step - loss: 3.0507 - accuracy: 0.3519
Epoch 5/200
1/1 [==============================] - 0s 13ms/step - loss: 2.9837 - accuracy: 0.3519
Epoch 6/200
1/1 [==============================] - 0s 17ms/step - loss: 2.9125 - accuracy: 0.3519
Epoch 7/200
1/1 [==============================] - 0s 12ms/step - loss: 2.8412 - accuracy: 0.4815
Epoch 8/200
1/1 [==============================] - 0s 10ms/step - loss: 2.7732 - accuracy: 0.4815
Epoch 9/200
1/1 [==============================] - 0s 12ms/step - loss: 2.7087 - accuracy: 0.4815
Epoch 10/200
1/1 [==============================] - 0s 14ms/step - loss: 2.6472 - accuracy: 0.4815
Epoch 11/200
1/1 [===

In [13]:
def prob_of_input_sentence(model, tokenizer, sentence):
    print ("Input Sentence:", sentence)
    encoded=tokenizer.texts_to_sequences([sentence])[0]
    encoded.insert(0, 0)
    encoded=np.array(encoded)
    encoded=np.reshape(encoded, newshape=(1, -1))
    prob = model.predict(encoded, verbose=0)
    probability=1
    for i in range (prob. shape[1]-1):
        probability = probability * prob[0, i, encoded[0, i+1]]
    print("Probability: ", probability)

In [14]:
prob_of_input_sentence(model, tokenizer, "I am Tirth Patel.")

Input Sentence: I am Tirth Patel.
Probability:  1.2854818300437701e-05


In [15]:
prob_of_input_sentence(model, tokenizer, "blasphemous")

Input Sentence: blasphemous
Probability:  0.00012432591756805778


In [16]:
prob_of_input_sentence(model, tokenizer, "Google Drive over GitHub")

Input Sentence: Google Drive over GitHub
Probability:  2.9634810912029133e-08


In [17]:
prob_of_input_sentence(model, tokenizer, "I am smart.")

Input Sentence: I am smart.
Probability:  0.00037150608505503237


In [18]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()


1122304/1115394 [==============================] - 0s 0us/step


In [19]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [20]:
tokenizer.texts_to_sequences(['Vicissitude'])

[[26, 6, 19, 6, 8, 8, 6, 3, 14, 13, 2]]

In [21]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [22]:
max_id = len(tokenizer.word_index)
max_id

39

In [23]:
dataset_size = tokenizer.document_count

In [24]:
dataset_size

1115394

In [25]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [26]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [27]:
n_steps = 100
window_length = n_steps + 1

In [28]:
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [29]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [30]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [31]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [32]:
dataset = dataset.prefetch(1)

In [33]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [34]:
class NvidiaUtilizationCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        text = !nvidia-smi
        text = text[9][60:65] + ' GPU utilization'
        print(text)

In [35]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))    
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')
steps_per_epoch = train_size // batch_size // n_steps

history = model.fit(dataset, epochs = 40, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback, NvidiaUtilizationCallback()])

Epoch 1/40
  0%  GPU utilization
313/313 [==============================] - 33s 8ms/step - loss: 2.9256
Epoch 2/40
 43%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 2.1227
Epoch 3/40
 36%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.8820
Epoch 4/40
 39%  GPU utilization
313/313 [==============================] - 2s 7ms/step - loss: 1.7226
Epoch 5/40
 27%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.5505
Epoch 6/40
 33%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.3864
Epoch 7/40
 36%  GPU utilization
313/313 [==============================] - 2s 7ms/step - loss: 1.3183
Epoch 8/40
 43%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.2648
Epoch 9/40
 41%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.2673
Epoch 10/40
 38%  GPU utilization
313/313 [=============================

In [36]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [43]:
X_new = preprocess(["I drink wate"])

In [44]:
Y_pred = np.argmax(model.predict(X_new), axis=-1)

In [45]:
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'r'

In [46]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [47]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [48]:
text_1 = complete_text("I", temperature=0.2)
text_1

'Ing surly his honour of an\nall this will he hath do'

In [49]:
text_2 = complete_text("I drink wate", temperature=1)
text_2

'I drink watery, in the timed\nsaur ne thus long an on comead,\nt'

In [57]:
text_3 = complete_text("I was walkin", temperature=0.5)
text_3

'I was walking this pleased strent of the ride in this the peac'